<h1 style="font-size: 74px">SFLKit: <br>A Workbench for Statistical Fault Localization</h1>

<div style="float: left; font-size:30px"><b>Marius Smytzek</b><br>marius.smytzek@cispa.de</div><div style="float: left;margin-left: 100px; font-size:30px"><b>Andreas Zeller</b><br>zeller@cispa.de</div>

[start](#Statistical-Fault-Localization)

In [1]:
from IPython.display import IFrame
src = f"https://www.youtube-nocookie.com/embed/qcnmi6PgrKg"
IFrame(src, 640, 360)

## What is SFLKit

SFLKit is a workbench for statistical fault localization. It comes with the fundamental concepts of statistical debugging and spectrum-based fault localization.

You can use SFLKit out-of-the-box by integrating its command-line interface `sfl.py` or as a library, as we do in this demonstration. We designed SFLKit to be highly configurable and expandable with novel concepts.

To install SFLKit execute

In [2]:
!pip install .

Processing /Users/marius/Desktop/work/projects/sflkit
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sflkit: filename=sflkit-0.2.6-py3-none-any.whl size=39297 sha256=aa8c1331ed527c0a89c0632b24b0ba146eab95760564d41188ef489542d5f516
  Stored in directory: /private/var/folders/09/pt1hglws43n7fh5521n6zyyh0000gn/T/pip-ephem-wheel-cache-1d3qc8oo/wheels/eb/95/8a/7bf6ee4d50417cf8e084552e4c6c5c1932d1e96f61309af921
Successfully built sflkit
  Attempting uninstall: sflkit
    Found existing installation: sflkit 0.2.6
    Uninstalling sflkit-0.2.6:
      Successfully uninstalled sflkit-0.2.6


In [3]:
import enum
import importlib
import inspect
import os
import shutil

from IPython.display import HTML

from sflkit.color import ColorCode
from sflkit import instrument_config, analyze_config
from sflkit.config import Config

## A faulty Program

First, we need a faulty program. We chose an implementation of the `middle(x, y, z)` function that returns the *middle* number of its three arguments. For example, `middle(1, 3, 2)` should return 2 because `1 < 2` and `2 < 3`. We introduced a fault in this implementation of `middle` that occurs in line 7 `m = y`. 

In [4]:
def middle(x, y, z):
    m = z
    if y < z:
        if x < y:
            m = y
        elif x < z:
            m = y  # bug
    else:
        if x > y:
            m = y
        elif x > z:
            m = x
    return m

Next, we introduce a class to capture test runs' results efficiently. The `TestResult` is an enum with two possible values, `PASS`and `FAIL`. `PASS` donates a passing test case and `FAIL` a failing one.

In [5]:
class TestResult(enum.Enum):
    
    def __repr__(self):
        return self.value
    
    PASS = 'PASS'
    FAIL = 'FAIL'

Now we implement a test function that takes the three arguments of `middle(x, y, z)` and an expected result. This test function compares the return of `middle(x, y, z)` with the desired value and returns `PASS` if they match and `FAIL` otherwise.

In [6]:
def test(function, x, y, z, expected):
    try:
        if function(x, y, z) == expected:
            return TestResult.PASS
        else:
            return TestResult.FAIL
    except BaseException:
        return TestResult.FAIL

def test_middle(x, y, z, expected):
    return test(middle, x, y, z, expected)

Let's check the results for some combinations of the numbers 1, 2, and 3. The expected value is in all cases 2.

In [7]:
test_middle(3, 2, 1, expected=2)

PASS

In [8]:
test_middle(3, 1, 2, expected=2)

PASS

In [9]:
test_middle(2, 1, 3, expected=2)

FAIL

As you can see, the result of `middle(2, 1, 3)` does not match the expected value 2. Hence, we found a failing test case

## Statistical Fault Localization

Statistical fault localization aims at detecting execution features that correlate with failures, such as whether individual lines are part of the execution.

With the example from above and the three test cases, we can run a statistical fault localization by marking which test case executes which lines:

<table>
<tr>
<td></td>
<td></td>
<td>

```python
middle(3, 2, 1)
```
            
</td>
<td>

```python
middle(3, 1, 2)
```
            
</td>
<td>

```python
middle(2, 1, 3)
```
            
</td>
</tr>
<tr>
<td>

```
1
2
3
4
5
6
7
8
9
10
11
12
13
```
            
</td>
<td>

```python
def middle(x, y, z):
    m = z
    if y < z:
        if x < y:
            m = y
        elif x < z:
            m = y  # bug
    else:
        if x > y:
            m = y
        elif x > z:
            m = x
    return m
```
            
</td>
<td>

```

       X
       X





       X
       X


       X
```
            
</td>
<td>

```

       X
       X
       X
       
       X






       X
```
         
</td>
<td>

```

       X
       X
       X
       
       X
       X





       X
```
            
</td>
</tr>
  
</table>

We can see here that line 7 gets only executed by the failing test and not by the passing tests, making it the most likely line to contain the fault, which we already know is true.

[next](#Configuring-SFLKit)

## Instrument the Program

Subsequently, we want to leverage SFLKit to find the location in the code that is most likely to include the fault.

Let us first get the source of our function and write it to a file so we have something to perform our instrumentation and analysis.

We leverage Python's `inspect` to get the source code.

In [10]:
source = inspect.getsource(middle)
print(source)

def middle(x, y, z):
    m = z
    if y < z:
        if x < y:
            m = y
        elif x < z:
            m = y  # bug
    else:
        if x > y:
            m = y
        elif x > z:
            m = x
    return m



We also define the file we write the source to and the python file we will work on, namely `middle.py` and `tmp.py`, respectively.

In [11]:
middle_py = 'middle.py'
tmp_py = 'tmp.py'

We write the source code to `middle.py`.

In [12]:
with open(middle_py, 'w') as fp:
    fp.write(source)

Let's update our test function to import the correct module and run the `middle(x, y, z)` from this module.

In [13]:
def test_middle_import(x, y, z, expected):
    from middle import middle
    return test(middle, x, y, z, expected)

We repeat the tests to check that our setup works with the import.

In [14]:
test_middle_import(3, 2, 1, expected=2), test_middle_import(3, 1, 2, expected=2), test_middle_import(2, 1, 3, expected=2)

(PASS, PASS, FAIL)

We produced the same results for the test cases, so it seems to work.

### Configuring SFLKit

The `Config` class provides comfortable access to `SFLKit` by defining the fundamental concepts we want to investigate.

We give some information for the config that we need to define. First, we need the path to the source we want to investigate, which we already have in `middle_py`. Next, we need an out, `tmp_py`. We also need:

The language of our subject is `'python'`.
Let's start with `'line'` as the predicates we want to investigate.
We define `'tarantula'` as our evaluation metric for the predicates, i.e., the similarity coefficient.
We also need a list of passing and failing tests used during the analysis.

In [15]:
language='python'
predicates='line'
metrics='Tarantula'
passing='event-files/0,event-files/1'
failing='event-files/2'

We define a function that gives as a `Config` object, so we do not need to create it manually every time we change something.

In [19]:
def get_config():
    return Config.create(path=middle_py, working=tmp_py, language=language, predicates=predicates, metrics=metrics, passing=passing, failing=failing)

Now we can define a function that instruments our subject. We leverage `SFLKit`'s `instrument_config()`, which takes a config we create with our defined `get_config()` and instruments the subject. We can also show the content of the instrumented python file with this function.

In [20]:
def instrument(out=True):
    instrument_config(get_config())
    if out:
        with open(tmp_py, 'r') as fp:
            print(fp.read())

Now we instrument our `middle.py` subject and check the results.

In [21]:
instrument()

import sflkitlib.lib


def middle(x, y, z):
    sflkitlib.lib.add_line_event('middle.py', 2, 0)
    m = z
    sflkitlib.lib.add_line_event('middle.py', 3, 1)
    if y < z:
        sflkitlib.lib.add_line_event('middle.py', 4, 2)
        if x < y:
            sflkitlib.lib.add_line_event('middle.py', 5, 3)
            m = y
        else:
            sflkitlib.lib.add_line_event('middle.py', 6, 4)
            if x < z:
                sflkitlib.lib.add_line_event('middle.py', 7, 5)
                m = y
    else:
        sflkitlib.lib.add_line_event('middle.py', 9, 6)
        if x > y:
            sflkitlib.lib.add_line_event('middle.py', 10, 7)
            m = y
        else:
            sflkitlib.lib.add_line_event('middle.py', 11, 8)
            if x > z:
                sflkitlib.lib.add_line_event('middle.py', 12, 9)
                m = x
    sflkitlib.lib.add_line_event('middle.py', 13, 10)
    return m



As you can see, the instrumentation added an import at the beginning to a lib that comes with `SFLKit`, cluing the execution of files together. Moreover, the instrumentation added a function call function of the lib in front of each executable line that tracks the executed lines.

## Get and Analyze Events

Now, we want to extract the events from the execution of tests. Therefore, we need to adjust our test execution function again because the shared library for tracking the events does not know when to start and stop. We need to reset this library before entering our `middle.py` and tell the library to dump the events after the function finishes.

In [32]:
def test_tmp(x, y, z, expected): 
    import tmp
    importlib.reload(tmp)
    tmp.sflkitlib.lib.reset()
    try:
        return test(tmp.middle, x, y, z, expected)
    finally:
        tmp.sflkitlib.lib.dump_events()
        del tmp

We define a path to write the generated event logs.

In [33]:
event_files = 'event-files'

Then, we need a function to generate the event log from the previous test cases. We change the environment variable `EVENTS_PATH` to the output path of the event log file before running each test.

In [34]:
def run_tests():
    if os.path.exists(event_files):
        shutil.rmtree(event_files)
    os.mkdir(event_files)
    os.environ['EVENTS_PATH'] = os.path.join(event_files, '0')
    test_tmp(3, 2, 1, expected=2)
    os.environ['EVENTS_PATH'] = os.path.join(event_files, '1')
    test_tmp(3, 1, 2, expected=2)
    os.environ['EVENTS_PATH'] = os.path.join(event_files, '2')
    test_tmp(2, 1, 3, expected=2)

With this, we can execute the tests and analyze the result with the help of `analyze_config()` from SFLKit.

In [35]:
def analyze():
    run_tests()
    return analyze_config(get_config())

Let's execute the tests and analyze the event logs for lines and the Tarantula metric.

In [36]:
results = analyze()

The results look something like this:

In [37]:
results

{'LINE': {'Tarantula': [[middle.py:7]:1.0,
   [middle.py:4, middle.py:6]:0.6666666666666666,
   [middle.py:3, middle.py:13, middle.py:2]:0.5,
   [middle.py:9, middle.py:10]:0.0]}}

This structure maps analysis objects and metrics to a list of sorted suggestions where the fault occurs.

Now, we can put all this together and produce a pretty output that shows us where the fault originates by leveraging `SFLKit`'s `ColorCode` object.

In [38]:
def sfl():
    instrument(out=False)
    results = analyze()
    code = ColorCode(results[predicates.upper()][metrics])
    return HTML(code.code(middle_py, source, color=True, suspiciousness=True))

In [39]:
sfl()

As you can see, the analysis indeed suggests the buggy line 7 as the most suspicious.

But what if lines are not enough to show the fault?

What if the metric we have chosen for evaluation is insufficient?

## Change the Analysis Object

Say you want to investigate different code elements, for instance, def-use pairs. We can do so by simply adjusting the `predicates` of the `Config` object. Keep in mind that `get_config()` automatically updates the `Config` object.

In [40]:
predicates='def_use'
sfl()

Now we have an analysis for def-use pairs. The bug occurs when `y` gets defined in line 1, used in line 7, and `m` gets defined in line 7 and used in line 13. So we found the data flow leading to the fault.

## Change the Metric

We can apply the same to the metric, for instance, if we want to investigate the Jaccard similarity coefficient.

In [41]:
metrics='Jaccard'
sfl()

We have an adjusted def-use pair analysis to the Jaccard coefficient with this result. You can compare it with the previous one and see the differences in the suspiciousness.

## From Spectra to Predicates

Up to this point, we have investigated spectra, but we could also examine predicates that need to hold, for instance, the conditions of branches.

In [42]:
predicates='branch'
metrics='IncreaseTrue'
sfl()

We can map these predicates to code locations and see what branch causes the fault.

# Thank You

## GitHub

https://github.com/uds-se/sflkit